## Activity and PPGR
Calculates ppgr auc, mets auc, average level, and activeMins metrics with a datetime of each meal for 2hr and 3hr timeframes

In [37]:
import numpy as np
import pandas as pd 
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

In [38]:
#read datafile for METs
x=1
datafile = pd.read_csv(f"C:\\Users\\namil\\Downloads\\CaM01-{str(x).zfill(3)}_Fitbit.csv")
time = datafile['mins'].to_numpy()
mets = datafile['METs'].to_numpy()
datafile['Date_Time'] = pd.to_datetime(datafile['Date_Time'])

time = time[~np.isnan(time)]
mets = mets[~np.isnan(mets)]

#read cgm datafiles
dexcomdata = pd.read_csv(f"C:\\Users\\namil\\Downloads\\CaM01-{str(x).zfill(3)}_CGM_Dexcom.csv")

In [39]:
def libreLinearInterpolation(cgm_df):
    # Convert the 'Date_Time' column to datetime type
    cgm_df['Date_Time'] = pd.to_datetime(cgm_df['Date_Time'])
    # Set the 'Date_Time' column as the index
    cgm_df.set_index('Date_Time', inplace=True)
    # Drop or fill missing values
    cgm_df.dropna(inplace=True)  # Drop rows with missing values
    # Fix duplicate error
    cgm_df = cgm_df.loc[~cgm_df.index.duplicated(), :]
    # Resample the DataFrame to 1-minute intervals and perform linear interpolation
    cgm_df_interpolated = cgm_df.resample('1T').interpolate(method='linear')
    # Reset the index back to a column
    cgm_df_interpolated.reset_index(inplace=True)
    return cgm_df_interpolated

def dexcomLinearInterpolation(raw_cgm_df):
    # Convert the 'Date_Time' column to datetime type
    raw_cgm_df['Date_Time'] = pd.to_datetime(raw_cgm_df['Date_Time'])
    # Set the seconds component to 0
    raw_cgm_df['Date_Time'] = raw_cgm_df['Date_Time'].apply(lambda dt: dt.replace(second=0))
    # Set the 'Date_Time' column as the index
    raw_cgm_df.set_index('Date_Time', inplace=True)
    # Resample the DataFrame to 1-minute intervals and perform linear interpolation
    interpolated_cgm_df = raw_cgm_df.resample('1T').interpolate(method='linear')
    interpolated_cgm_df
    # Reset the index back to a column
    interpolated_cgm_df.reset_index(inplace=True)
    return interpolated_cgm_df

def readData(x):
    #get globals
    global datafile
    global time
    global mets
    global libredata
    global dexcomdata 
    
    #read datafile for METs
    datafile = pd.read_csv(f"C:\\Users\\namil\\Downloads\\CaM01-{str(x).zfill(3)}_Fitbit.csv")
    time = datafile['mins'].to_numpy()
    mets = datafile['METs'].to_numpy()
    datafile['Date_Time'] = pd.to_datetime(datafile['Date_Time'])

    time = time[~np.isnan(time)]
    mets = mets[~np.isnan(mets)]

    #read cgm datafiles
    #libredata = libreLinearInterpolation(pd.read_csv(f"C:\\Users\\namil\\Downloads\\CaM01-{str(x).zfill(3)}_CGM_Libre.csv")) 
    dexcomdata = dexcomLinearInterpolation(pd.read_csv(f"C:\\Users\\namil\\Downloads\\CaM01-{str(x).zfill(3)}_CGM_Dexcom.csv"))

### Calculate metrics for all the meal times from a spreadsheet
can export to csv or print yay

In [40]:
def activityMetrics(timestring):
    tuples = zip(time,mets)
    arr = np.asarray(list(tuples))

    #find start time
    #target_time = datetime.strptime(timestring, '%m/%d/%Y %H:%M')
    target_time = datetime.strptime(timestring, '%Y-%m-%d %H:%M:%S')
    theday = datafile[datafile['Date_Time'].dt.date == target_time.date()]
    starttime = theday.iloc[0]['Date_Time']
    minutes = int((target_time - starttime).total_seconds() / 60 + theday.iloc[0]['mins'])

    timeframe = 180
    #filter to time interval
    filtered = arr[arr[:,0] <= minutes + timeframe]
    filtered = filtered[filtered[:,0] >= minutes]

    #calculate iauc for that time interval
    activity = filtered[:,1]
    area = auc(filtered[:,0], activity)
    auc3.append(area)
    avg3.append(np.average(activity)/10)
    activeMin3.append(len(activity[activity > 20]))


    filtered = filtered[filtered[:,0] <= minutes + 120]
    activity = filtered[:,1]
    area = auc(filtered[:,0], activity)
    auc2.append(area)
    avg2.append(np.average(activity)/10)
    activeMin2.append(len(activity[activity > 20]))

In [55]:
peakheight = []
peaktime = []
pd40 = []
startbg = []
iauc = []

def iaucAll(timestring, threehr): 
    datafile1 = dexcomdata
    
    time = datafile1['mins'].to_numpy()
    bg = datafile1['BG'].to_numpy()
    time = time[~np.isnan(time)]
    bg = bg[~np.isnan(bg)]
    datafile1['Date_Time'] = pd.to_datetime(datafile1['Date_Time'])
    
    tuples = zip(time,bg)
    arr = np.asarray(list(tuples))
    
    #target_time = datetime.strptime(timestring, '%m/%d/%Y %H:%M')
    target_time = datetime.strptime(timestring, '%Y-%m-%d %H:%M:%S')
    theday = datafile1[datafile1['Date_Time'].dt.date == target_time.date()]
    
    try:
        if(len(theday) == 0):
            raise ValueError("Data for this date is not available")
        starttime = theday.iloc[0]['Date_Time']
        minutes = int((target_time - starttime).total_seconds() / 60 + theday.iloc[0]['mins'])
    
        #filter to time interval
        filtered = arr[arr[:,0] >= minutes]
        filtered = filtered[filtered[:,0] <= minutes + 180]
        cgmdata = filtered[:,1]
        
        area = auc(filtered[:,0], cgmdata)
        
        #check that start and end pts are available (will look for a better way later)
        checkstart = filtered[filtered[:,0] == minutes]
        checkend = filtered[filtered[:,0] == minutes + 180]
        
        if len(checkstart) == 0 or len(checkend) == 0:
             raise ValueError('Endpoint not available')
                        
        peak = np.max(cgmdata)
        ptime = filtered[np.argmax(cgmdata), 0]
        pd4 = len(cgmdata[cgmdata > peak*.6])
        start = cgmdata[0]
        iaucval = area - start*180
       
    except ValueError:
        area = "error"
        peak = "error"
        pd4 = "error"
        ptime = "error"
        start = "error"
        iaucval = "error"
        
    threehr.append(area)
    peakheight.append(peak)
    pd40.append(pd4)
    peaktime.append(ptime)
    startbg.append(start)
    iauc.append(iaucval)


In [53]:
#calculate one person
dtwo = []
dthree = []
auc2 = []
auc3 = []
activeMin2 = []
activeMin3 = []
avg2 = []
avg3 = []
peakheight = []
peaktime = []
startbg = []
iauc = []

participant = 3
mldata = pd.read_excel(f"C:\\Users\\namil\\Downloads\\{str(participant).zfill(3)}.xlsx").dropna()
mldata['Meal Time'] = mldata['Meal Time'].to_numpy()
readData(participant)
    
for i in mldata['Meal Time']:
    print('\n' + str(i))
    activityMetrics(str(i))
    iaucAll(str(i), dthree)

df = pd.DataFrame({'Meal Time': mldata['Meal Time'], 'snack in 3hrs': mldata['snack'], 'Meal Type': mldata['Meal Type'],
                   #'dexcom 2hr auc': dtwo, 'mets 2 hr auc': auc2, '2 hr activeMin': activeMin2, 'mets 2 hr avg': avg2, 'libre 2 hr auc': twohr,
                   'dexcom 3hr auc':dthree, 'mets 3 hr auc': auc3, '3 hr activeMin': activeMin3, 'mets 3 hr avg': avg3, 'peak': peakheight, 
                   'calories':mldata['Calories'], 'carbs':mldata['Carbs'], 'protein':mldata['Protein'],  'fat':mldata['Fat']})
df.to_csv(f'allmetrics{participant}')
print("done!")


2021-09-29 12:09:00

2021-09-29 19:38:00

2021-09-30 10:17:00

2021-09-30 14:25:00

2021-09-30 20:04:00

2021-10-01 08:27:00

2021-10-01 12:33:00

2021-10-01 20:53:00

2021-10-02 09:35:00

2021-10-02 13:33:00

2021-10-02 19:57:00

2021-10-03 09:12:00

2021-10-03 14:06:00

2021-10-03 20:05:00

2021-10-04 08:14:00

2021-10-04 16:30:23

2021-10-04 20:15:00

2021-10-05 09:07:00

2021-10-05 13:24:00

2021-10-05 20:02:00

2021-10-06 09:15:00

2021-10-06 13:34:00

2021-10-06 20:12:00

2021-10-07 10:08:00

2021-10-07 14:07:00

2021-10-07 19:46:00

2021-10-08 08:13:00

2021-10-08 12:45:00

2021-10-08 19:46:00
done!


In [ ]:
#generate dataset for all participants
participants = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29]
for x in participants:
    dthree = []
    auc3 = []
    activeMin3 = []
    avg3 = []
    peakheight = []
    peaktime = []
    pd40 = []
    startbg = []
    iauc = []

    print("\n" + f"participant number {x}:")
    mldata = pd.read_excel(f"C:\\Users\\namil\\Downloads\\{str(x).zfill(3)}.xlsx").dropna()
    mldata['Meal Time'] = mldata['Meal Time'].to_numpy()
    readData(x)

    for i in mldata['Meal Time']:
        print(str(i))
        activityMetrics(str(i))
        iaucAll(str(i), dthree)

    df = pd.DataFrame({'Meal Time': mldata['Meal Time'], 'snack in 3hrs': mldata['snack'], 'Meal Type': mldata['Meal Type'],
                   #'dexcom 2hr auc': dtwo, 'mets 2 hr auc': auc2, '2 hr activeMin': activeMin2, 'mets 2 hr avg': avg2,
                   'peakheight': peakheight, 'peaktime': peaktime, 'peakduration_40': pd40, 'startbg': startbg, 'iauc': iauc,
                   'dexcom 3hr auc':dthree, 'mets 3 hr auc': auc3, '3 hr activeMin': activeMin3, 'mets 3 hr avg': avg3,
                   'calories':mldata['Calories'], 'carbs':mldata['Carbs'], 'protein':mldata['Protein'],  'fat':mldata['Fat']})
    df.to_csv(f'allmetrics{x}')
    
print("done :)")


participant number 1:
2021-09-18 14:23:00
2021-09-18 20:48:00
2021-09-19 08:18:00
2021-09-19 12:00:00
2021-09-19 20:10:00
2021-09-20 09:07:00
2021-09-20 14:51:00
2021-09-20 19:52:00
2021-09-21 09:17:00
2021-09-21 12:59:00
2021-09-21 19:42:00
2021-09-22 09:22:00
2021-09-22 13:16:00
2021-09-22 21:05:00
2021-09-23 08:54:00
2021-09-23 12:25:00
2021-09-23 19:56:00
2021-09-23 21:19:00
2021-09-24 10:07:00
2021-09-24 14:12:00
2021-09-24 19:36:00
2021-09-25 09:58:00
2021-09-25 13:26:00
2021-09-25 20:16:00
2021-09-26 09:35:00
2021-09-26 13:46:00
2021-09-26 18:26:00
2021-09-27 09:27:00
2021-09-27 12:47:00
2021-09-27 20:00:00
[1029.0, 1284.0, 1974.0, 2204.0, 2754.0, 3459.0, 3934.0, 4249.0, 4924.0, 5164.0, 5682.0, 6369.0, 6569.0, 7194.0, 7769.0, 7994.0, 8544.0, 8609.0, 9284.0, 9549.0, 9994.0, 10704.0, 10889.0, 11359.0, 12169.0, 12384.0, 12639.0, 13564.0, 13774.0, 14339.0]

participant number 2:
2021-09-26 10:03:00
2021-09-26 13:27:00
2021-09-26 17:14:00
2021-09-27 10:33:00
2021-09-27 14:24:00
2021